In [1]:
import brightway2 as bw
import bw2calc as bc
import bw2data as bd
import numpy as np
import pandas as pd
import presamples as ps
import scipy
import scipy.stats as stats

In [ ]:
from code/filtering_markets import recursive_calc  
from code/filtering_markets import get_lca

In [3]:
bw.projects.set_current("UK-wood-clca")

cutoff391 = bw.Database('cutoff-3.9.1')

methods = [
      ('ReCiPe 2016 v1.03, midpoint (H)', 'climate change', 'global warming potential (GWP1000)'), 

]

In [4]:
import re
def clean_name(name):
    # Replace any non-alphanumeric character (including spaces) with an underscore
    clean_name = re.sub(r'[^a-zA-Z0-9]', '_', name)
    return clean_name

In [ ]:
fus = []

names = []
for v in range(1):
    result =   cutoff391.get('1cf8a7ab4ea4bf765ee0119b1f20d570') #cutoff391.random(v)  # Fetch the result once 
    name = str(result)  # Convert it to a string if needed
    
    lca = bw.LCA({result:1}, method=methods[0])
    lca.lci()
    lca.lcia()
    
    if lca.score == 0 or lca.score < 0:
        continue
  
    # Convert name to lowercase for case-insensitive matching
    if 'electricity' in name.lower() or 'waste' in name.lower() or 'treatment' in name.lower() or 'land' in name.lower() or 'cutoff' in name.lower():
        continue  # Skip this iteration if the name contains the unwanted words
    names.append(clean_name(name))
    
    fus.append(result[1]) 

In [ ]:
numbers = fus
fus = [cutoff391.get({num}) for num in numbers]

In [11]:
fu = pd.read_excel(r"run.xlsx", sheet_name='to_run_local')

In [12]:
numbers = fu['indices'].tolist()
fus = [cutoff391.get({num}) for num in numbers]

In [13]:
names = fu['names'].tolist()

In [18]:
names

['_chromium_production___kilogram__RoW__None_']

In [19]:
%%time 
for i in range(len(fus)):
    markets = recursive_calc(fus[i], methods, cutoff= 5e-5, max_calc =1000000000000, output_dir= names[i])

Non-stochastic LCA score: 30.61757540791155
cutoff: 5e-05, max_calc: 1000000000000
TECHNOSPHERE (21238, 21238) filtering resulted in 12084 of 266440 exchanges and took 21497 iterations in 111.79 seconds.
CLCA analysis complete. Results saved to _chromium_production___kilogram__RoW__None_
CPU times: total: 15min 26s
Wall time: 2min 25s
